In [1]:
import geopandas as gp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import mplleaflet
# get rid of stupid warning
pd.options.mode.chained_assignment = None

In [2]:
# PUMAs shape files
pumas = gp.read_file('cb_2016_06_puma10_500k/cb_2016_06_puma10_500k.shp')
# read in household PUMS data
hca = pd.read_csv('ss16hca.csv')

## Get Internet Access Data

In [3]:
# extract samples corresponding to each county
alameda_puma = np.arange(101, 111)
cc_puma = np.arange(1301, 1310)
sf_puma = np.arange(7501, 7508)
# extract household data
alameda = hca.loc[hca.PUMA.isin(alameda_puma)]
cc = hca.loc[hca.PUMA.isin(cc_puma)]
sf = hca.loc[hca.PUMA.isin(sf_puma)]

In [4]:
def extend_low_income(thres, upper_N=16):
    extras = (np.arange(9, upper_N) - 8) * thres[3] * 0.08 + thres[-1]
    thres = np.append(thres, extras)
    return thres

In [5]:
alameda_low_income_thres = np.array([62750, 71700, 80650, 89600, 96800, 103950, 111150, 118300])
cc_low_income_thres = np.array([62750, 71700, 80650, 89600, 96800, 103950, 111150, 118300])
sf_low_income_thres = np.array([82200, 93950, 105700, 117400, 126800, 136200, 145600, 155000])
# extend the low income thresholds for households of size >8
alameda_low_income_thres = extend_low_income(alameda_low_income_thres)
cc_low_income_thres = extend_low_income(cc_low_income_thres)
sf_low_income_thres = extend_low_income(sf_low_income_thres)

In [6]:
def get_low_income(df, low_income_thresholds):
    # use FINCP or HINCP?
    # remove NaN household incomes
    trimmed = df.loc[df.FINCP.notna()]
    low_income = trimmed.loc[trimmed.FINCP < low_income_thresholds[trimmed.NP - 1]]
    return low_income

In [7]:
alameda_low_income = get_low_income(alameda, alameda_low_income_thres)
cc_low_income = get_low_income(cc, cc_low_income_thres)
sf_low_income = get_low_income(sf, sf_low_income_thres)

In [8]:
def calculate_percent_internet(table, weighted=False):
    if weighted:
        weighted_sum = table.groupby('PUMA').WGTP.sum()
        no_internet = table.loc[table.ACCESS == 3].groupby('PUMA').WGTP.sum()
        percent_no_internet = 100 * no_internet/weighted_sum
    else:
        total_count = table.groupby('PUMA').count()
        no_internet = table.loc[table.ACCESS==3].groupby('PUMA').count()
        percent_no_internet = 100 * (no_internet/total_count).iloc[:, 0]
    return percent_no_internet.fillna(value=0)

def calculate_percent_laptop(table, weighted=False):
    if weighted:
        weighted_sum = table.groupby('PUMA').WGTP.sum()
        no_internet = table.loc[table.LAPTOP == 2].groupby('PUMA').WGTP.sum()
        percent_no_internet = 100 * no_internet/weighted_sum
    else:
        total_count = table.groupby('PUMA').count()
        no_internet = table.loc[table.LAPTOP == 2].groupby('PUMA').count()
        percent_no_internet = 100 * (no_internet/total_count).iloc[:, 0]
    return percent_no_internet

In [9]:
# internet
alameda_internet = calculate_percent_internet(alameda_low_income, weighted=True)
cc_internet = calculate_percent_internet(cc_low_income, weighted=True)
sf_internet = calculate_percent_internet(sf_low_income, weighted=True)
# laptops
alameda_laptop = calculate_percent_laptop(alameda_low_income, weighted=False)
cc_laptop = calculate_percent_laptop(cc_low_income, weighted=False)
sf_laptop = calculate_percent_laptop(sf_low_income, weighted=False)

In [10]:
# grab geoframe info 
alameda_gf = pumas.loc[pumas.NAME10.str.contains('Alameda County')].sort_values('PUMACE10')
cc_gf = pumas.loc[pumas.NAME10.str.contains('Contra Costa County')].sort_values('PUMACE10')
sf_gf = pumas.loc[pumas.NAME10.str.contains('San Francisco County')].sort_values('PUMACE10')

In [11]:
# toss internet in geodataframe
alameda_gf['internet'] = np.array(alameda_internet)
cc_gf['internet'] = np.array(cc_internet)
sf_gf['internet'] = np.array(sf_internet)
# toss laptop in geodataframe
alameda_gf['laptop'] = np.array(alameda_laptop)
cc_gf['laptop'] = np.array(cc_laptop)
sf_gf['laptop'] = np.array(sf_laptop)

In [12]:
# concatenate all geodataframes
bay_area_gf = pd.concat([alameda_gf, cc_gf, sf_gf])

In [13]:
ax = bay_area_gf.plot(edgecolor='black', alpha=0.75, figsize=(20, 10), column='internet', cmap='Blues', scheme='quantiles')
mplleaflet.display(fig=ax.figure)

In [14]:
ax = bay_area_gf.plot(edgecolor='black', alpha=0.75, figsize=(20, 10), column='laptop', cmap='Blues', scheme='quantiles')
mplleaflet.display(fig=ax.figure)